Import Stuff

In [ ]:
import ssl
import re


from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from umap import UMAP

from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datetime import datetime
from textblob import TextBlob
from datasets import load_dataset

import pickle
import importlib
import functions
import requests





In [ ]:
from functions import *
importlib.reload(functions)

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

parties = ["CDU", "SPD", "FDP", "AFD", "LEFT", "GREENS"]
file_path = "C:/Users/Jacob/OneDrive/uni/MA WiSoz/Semester III/Computational Social Sciences/foPra/data/"


# Open the file in read mode
with open("API Keys/DeepL.txt", 'r', encoding='utf-8') as file:
    # Read the contents of the file
    api_key_deepl = file.read()

url = 'https://api-free.deepl.com/v2/translate'
df_manifesto = pd.read_csv("data/parties/Manifestos_tokenized.csv")



DeepL API

In [ ]:
params = {
    'auth_key': api_key,
    'text': 'Hello, world!',
    'target_lang': 'DE'  # Translate to German
}

In [ ]:
# Make the API request
response = requests.post(url, data=params)


In [ ]:
# Check if the request was successful
if response.status_code == 200:
    result = response.json()
    translated_text = result['translations'][0]['text']
    print(f'Translated text: {translated_text}')
else:
    print(f'Error: {response.status_code}')
    print(response.text)

Edit Text documents

In [ ]:
lemmatizer = WordNetLemmatizer()

defining different models
    

In [ ]:
umap_model = UMAP(
    n_neighbors=50, 
    # n_components=5, 
    # min_dist=0.0, 
    metric='cosine', 
    random_state=42
    )

In [ ]:
embedding_model_en = "all-MiniLM-L6-v2"
embedding_model_multilingual1 = "paraphrase-multilingual-mpnet-base-v2"
embedding_model_multilingual2 = "distiluse-base-multilingual-cased-v1"

In [ ]:
representation_model = KeyBERTInspired()
cluster_model = KMeans(n_clusters=30)

In [ ]:
hdbscan_model = HDBSCAN(
    min_cluster_size=50, 
    # min_samples=10,
    metric='euclidean', 
    prediction_data=True
    )



In [ ]:
seed_topic_list = [["steuergeld", "ausgaben", "staatsausgaben", "staatshaushalt", "schulden", "schuldenbremse", "staatsschulden"],
                   ['sozial wohnungsbau', 'sozialen wohnungsbaus', 'sozialen wohnungsbau', 'sozialwohnungen', 'wohnungsbauförderung', 'mietwohnungen', 'wohnungsbaus', 'bezahlbar wohnungen', 'eigentumswohnungen', 'wohnungsbau'],
                   ['öffentlich investitionen', 'öffentlichen investitionen', 'investitionen öffentlichen', 'investitionen', 'investitionsquot', 'investitionsstau', 'investitionszulag', 'investitionsmittel', 'privat investitionen', 'investitionsanr']]

Initialisiere und trainiere das BERTopic-Modell


In [ ]:

# Ensure the 'text' column contains only strings and handle NaN values
df_manifesto["text"] = df_manifesto["text"].astype(str).fillna("")

topic_model = BERTopic(
    language="multilingual"
    , n_gram_range=(1,2)
    # , min_topic_size=20
    # , top_n_words=20
    # , representation_model=representation_model
    , embedding_model=embedding_model_multilingual2
    , umap_model=umap_model
    # , hdbscan_model=hdbscan_model
    , seed_topic_list=seed_topic_list
    )

#topic_model = BERTopic(nr_topics=100, calculate_probabilities=True, embedding_model="all_MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(df_manifesto["text"])



In [ ]:
df_manifesto["topic"] = topics

In [ ]:
df_manifesto.to_pickle('Manifesto_final.pkl')

In [ ]:
topic_model.get_topic_info()


In [ ]:
len(topic_model.get_topic(25))

In [ ]:

# Erstelle das `topics_over_time`-Objekt
topics_over_time = topic_model.topics_over_time(df_manifesto["text"], df_manifesto["date"])
#topic_model = BERTopic(representation_model=KeyBERTInspired())


In [ ]:
# Save topics_over_time to a file
with open("topics_over_time.pkl", "wb") as f:
    pickle.dump(topics_over_time, f)

In [ ]:

# Method 1 - safetensors
topic_model.save(file_path[:-5], serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model_multilingual2)

# Method 2 - pytorch
#topic_model.save(file_path, serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# Method 3 - pickle
#topic_model.save("bertopic_model.pkl")



In [ ]:
# Save the topics to a pickle file
with open('topics.pkl', 'wb') as f:
	pickle.dump(topics, f)

# Save the probabilities to a pickle file
with open('probs.pkl', 'wb') as f:
	pickle.dump(probs, f)